In [1]:
import pandas as pd
import numpy as np
from collections import Counter
from datetime import datetime

from tqdm.notebook import tqdm

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [2]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
tags = pd.read_csv('tags.csv')

In [3]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [5]:
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [6]:
def change_string(s):
    return ' '.join(s.replace(' ', '').replace('-', '').split('|'))

In [7]:
movie_genres = [change_string(g) for g in movies.genres.values]

genres_tfidf = TfidfVectorizer()
X_train_genres_tfidf = genres_tfidf.fit_transform(movie_genres)

In [8]:
movies_with_tags = movies.join(tags.set_index('movieId'), on='movieId')
movies_with_tags.tag.unique()
movies_with_tags.dropna(inplace=True)

tag_strings = []
for movie, group in tqdm(movies_with_tags.groupby('movieId')):
    tag_strings.append(' '.join([change_string(s) for s in group.tag.values]))
    
tags_tfidf = TfidfVectorizer()
X_train_tags_tfidf = tags_tfidf.fit_transform(tag_strings)

  0%|          | 0/1572 [00:00<?, ?it/s]

Система оценки будет включать 2 группы параметров
1. описание фильмов, т.е. векторное представление жанров, тегов, а также средний рейтинг фильма
2. средний и медианный рейтинг, который дает каждый пользователь, а также стандартное отклонение рейтинга в разрезе пользователя

целевая метрика - рейтинг, который пользователь поставил фильму

In [9]:
# создадим DF с tf-idf для жанров
# genres_tfidf - tfidf для жанров

data = {}
for m in movies[["movieId", "genres"]].values:
    movie_id = m[0]
    genres_string = change_string(m[1])
    genres_tfidf_values = genres_tfidf.transform([genres_string]).toarray()[0]
    data[movie_id] = genres_tfidf_values

movies_params_genres = pd.DataFrame.from_dict(data, orient='index')
movies_params_genres.index.name = 'movieId'
movies_params_genres.rename(lambda x: f'genre_{x}', axis=1, inplace=True)

In [10]:
# создадим DF с tf-idf для тегов
# tags_tfidf - tfidf для тэгов

data = {}
for movie, group in tqdm(movies_with_tags[["movieId", "tag"]].groupby('movieId')):
    tags_string = " ".join([t for t in group["tag"].values])
    tags_string = change_string(tags_string)
    tags_tfidf_values = tags_tfidf.transform([tags_string]).toarray()[0]
    data[movie] = tags_tfidf_values
movies_params_tags = pd.DataFrame.from_dict(data, orient='index')
movies_params_tags.index.name = 'movieId'
movies_params_tags.rename(lambda x: f'tags_{x}', axis=1, inplace=True)

  0%|          | 0/1572 [00:00<?, ?it/s]

In [11]:
# объеденим оба DF
movies_params = movies_params_genres.join(movies_params_tags, on="movieId", lsuffix="g", rsuffix="t")
# уберем пропуски тегов. заменим пропуски на 0, чтобы не потерять оценки по фильмам
movies_params = movies_params.fillna(0)

In [13]:
# рассчитаем средний рейтинг фильма
movies_rating_avg = ratings[["movieId", "rating"]].groupby("movieId")["rating"].mean()
# добавим средний рейтинг фильма данные с параметрами
movies_params = movies_params.join(movies_rating_avg)
# для фильмов без рейтинга, заменим значения на средний рейтинг по выборке
movies_params['rating'].fillna((movies_params['rating'].mean()), inplace=True)
# movies_params['rating'].dropna(inplace=True)
movies_params.rename(columns={'rating': 'movie_rating_avg'}, inplace=True)

In [14]:
# добавляем к массиву данных о фильмах - пользователя и его оценку
# оценка будет нашей целевой переменной
# индексом станет комбинация фильма и пользователя
movies_params = ratings[["movieId", "userId", "rating"]].join(movies_params, on="movieId")
# удаляем фильмы где пользователь не поставил оценку
movies_params.dropna(inplace=True)
movies_params["movie-user"] = movies_params['movieId'].apply(str) +"-"+movies_params['userId'].apply(str)
movies_params.drop(columns=["movieId"], inplace=True)
movies_params.reset_index(drop=True)
movies_params.set_index("movie-user", inplace=True)

In [15]:
# определим средний, медианный рейтинг, стд.отклонение рейтинга, который ставит каждый пользователь
# эти параметры будут описывать нашего пользователя
user_rating_avg = ratings[["userId", "rating"]].groupby("userId").agg(
    UserMeanRating=('rating', np.mean),
    UserMedianReting=('rating', np.median),
    UserVarianceRating=('rating', np.std)
)
# добавляем данные о пользователе в массив значений
movies_params = movies_params.join(user_rating_avg, on="userId")

In [16]:
all_data = movies_params.copy()
all_data.reset_index(drop=True,inplace=True)

# userId = 1
# y = all_data[all_data["userId"]==userId]["rating"]
# X = all_data[all_data["userId"]==userId].drop(columns=["rating", "userId"])
y = all_data["rating"]
X = all_data.drop(columns=["rating", "userId"])

X_scaler = MinMaxScaler()
X = X_scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

reg = LinearRegression(positive=True).fit(X_train, y_train)

In [19]:
# расчет ошибки по тестовому массиву данных
y_pred = reg.predict(X_test)
mean_squared_error(y_test, y_pred)

0.664382396996206

In [20]:
# пример ошибки по пользователю
userId = 1

y = all_data[all_data["userId"]==userId]["rating"]
X = all_data[all_data["userId"]==userId].drop(columns=["rating", "userId"])
X = X_scaler.transform(X)

y_pred = reg.predict(X)
mean_squared_error(y, y_pred)

0.5398622139587878